# Importando e tratando a base de dados

In [ ]:
import requests
from io import StringIO

# parametros da request (github info)
owner = 'danplevs'
token = 'd9d7f4c1c64d6ec82b7c086c9720698d21ab9f48'
repo = 'geohmount'
path = '/trajs/sb/resultado_manipulado.csv'

# enviando o request
r = requests.get(f'https://api.github.com/repos/{owner}/{repo}/contents/{path}',
    headers={
        'accept': 'application/vnd.github.v3.raw',
        'authorization': f'token {token}'
            }
    )

# convertendo a string em objeto stringIO
string_io_obj = StringIO(r.text)

In [ ]:
import pandas as pd

trajs = trajs = pd.read_csv(string_io_obj)

In [ ]:
trajs.head()

In [ ]:
trajs = trajs.drop('Unnamed: 0', 1)
trajs.head()

# Análise exploratória dos dados

## Proporções

In [ ]:
trajs_659 = trajs.query('altitude == 659')

<h3>O gráfico de rosetas nos dá uma boa noção das principais direções dos ventos que chegam ao Soberbo provocando chuva, mas quais são as proporções dessas direções?

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

prop_direcoes = trajs_659['direcao_cat'].value_counts(normalize=True) * 100

fig = plt.figure(figsize=(14, 8))
sns.set_context('talk')
sns.barplot(x=prop_direcoes.index, y=prop_direcoes, palette='flare_r')
plt.ylabel('%')
plt.title('Direções')
# atualiza a figura
# fig.savefig('proporcao_direcoes.png') 

<h3> E das velocidades e chuvas (soma por evento)?

In [ ]:
velocidades = ['0-5 km/h', '5-10 km/h', '10-15 km/h', '15-20 km/h', '20-25 km/h', '25-30 km/h', '30-35 km/h', '35-40 km/h','40-45 km/h']

prop_velocidades = trajs_659['velocidade_cat'].value_counts(normalize=True)* 100
prop_velocidades = prop_velocidades.reindex(index=velocidades)

In [ ]:
fig = plt.figure(figsize=(20, 8))
sns.set_context('talk')
sns.barplot(x=prop_velocidades.index, y=prop_velocidades, palette='magma_r')
plt.ylabel('%')
plt.title('Velocidades')
# atualiza a figura
# fig.savefig('proporcao_velocidades')

In [ ]:
chuvas = ['5-20 mm', '20-35 mm', '35-50 mm', '50-65 mm', '65-80 mm', '80-95 mm', '95-110 mm', '110-125 mm', '125-130 mm']

prop_chuvas = trajs_659['evt_sum_rain_cat'].value_counts(normalize=True).sort_index() * 100
prop_chuvas = prop_chuvas.reindex(index=chuvas)

In [ ]:
fig = plt.figure(figsize=(18, 8))
sns.barplot(x=prop_chuvas.index, y=prop_chuvas, palette='Blues')
plt.ylabel('%')
plt.title('Precipitação')
# atualiza a figura
# fig.savefig('proporcao_chuva.png')

## Precipitação

<h3> Mas qual será a distribuição das precipitações em cada direção? Será que existe uma direção que tende a provocar chuvas fortes ou enxurradas?

In [ ]:
fig = plt.figure(figsize=(16, 8))
sns.boxplot(data=trajs_659, x='direcao_cat', y='evt_sum_rain')
plt.ylabel('Soma de precipitação do evento (mm)'), plt.xlabel('Direção')
# atualiza a figura
# fig.savefig('chuva_direcoes_boxplot.png')

In [ ]:
principais_direcoes = trajs_659.query('direcao_cat == ["SSW", "S", "E", "ESE"]')
fig = plt.figure(figsize=(12, 8))
sns.boxplot(data=principais_direcoes, x='direcao_cat', y='evt_sum_rain')
plt.ylabel('Soma de precipitação do evento (mm)'), plt.xlabel('Direção')
# atualiza a figura
#fig.savefig('principais_direcoes.png')